In [29]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly.graph_objs import*
import plotly.graph_objs as go
import math
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)



In [127]:
electric = pd.read_csv('~/Downloads/Full+Eletric+Interval+042016.csv')
electric['Datetime'] = pd.to_datetime(electric['Date'] + ' ' + electric['Start Time'])

In [3]:
pc = pd.read_excel('~/Downloads/SIMS_Property_List_4-6-2016_Revised.xlsx')

In [109]:
weather = pd.read_csv('~/Downloads/weather_prepared_final.csv')
weather.DATE = pd.to_datetime(weather.DATE)

### Temperature

In [121]:
temperature = weather.drop(['HUMIDITY', 'LON', 'LAT', 'STATION'], 1).iloc[1::4, :]

### Daily average temperature

In [122]:
daily_temperature = temperature
daily_temperature.DATE = temperature.DATE.dt.date
daily_temperature = daily_temperature.groupby(by='DATE').mean()

TEMPERATURE
DATE                   
2014-12-02    29.241667
2014-12-03    31.891667
2014-12-04    32.304167
2014-12-05    36.420833
2014-12-06    38.564583
2014-12-07    34.664583
2014-12-08    36.297917
2014-12-14    49.343750
2014-12-15    45.783333
2014-12-16    40.997917
2014-12-17    26.604167
2014-12-18    26.627083
2014-12-19    31.547917
2014-12-20    33.125000
2014-12-21    35.156250
2014-12-22    35.816667
2014-12-23    47.008333
2014-12-24    39.833333
2014-12-25    39.112500
2014-12-26    45.731250
2014-12-27    42.464583
2014-12-31    13.487500
2015-01-01    23.854167
2015-01-02    29.258333
2015-01-03    34.429167
2015-01-04    24.308333
2015-01-05     4.922222
2015-01-06     8.975000
2015-01-07     2.764865
2015-01-08    10.596429
...                 ...
2016-09-23    70.935417
2016-09-24    68.389583
2016-09-25    73.039583
2016-09-26    64.685417
2016-09-27    62.289583
2016-09-28    56.820833
2016-09-29    63.500000
2016-09-30    62.687500
2016-10-01    61.431250
2016-10-02    61.187500
2016-10-03    62.408333
2016-10-04    66.260417
2016-10-05    72.381250
2016-10-06    67.995833
2016-10-07    64.308333
2016-10-08    53.750000
2016-10-09    56.843750
2016-10-10    60.270833
2016-10-11    66.316667
2016-10-12    60.839583
2016-10-13    49.527083
2016-10-14    56.531250
2016-10-15    63.177083
2016-10-16    68.041667
2016-10-17    74.618750
2016-10-18    70.077083
2016-10-19    61.950000
2016-10-20    55.918750
2016-10-21    48.079167
2016-10-22    50.533333

[683 rows x 1 columns]

### Usage by day

In [129]:
electric_date_full = electric.copy()
electric_date_full.Datetime = electric_date_full.Datetime.dt.date

In [130]:
electric_day = electric_date_full.drop('Temperature', 1).groupby(by=['Meter', 'Datetime']).sum()

### Codes

In [4]:
codes = pd.Series([b[0] for b in electric['Meter'].str.split()])
unicodes = codes.unique()
unicodes = pd.Series(unicodes)

### Ages

In [5]:
agelst = []

for i in unicodes:
    agelst.append(pc[pc['Property Code'].isin([i])]['Year Constructed'])
age = pd.DataFrame(agelst).sum(axis=1)
age = age.reset_index(drop=True)

### Areas

In [6]:
arealst = []
for b in unicodes:
    arealst.append(pc[pc['Property Code'].isin([b])]['FCIM Gross Area'])
area = pd.DataFrame(arealst).sum(axis=1)
area = area.reset_index(drop=True)

### Cumulative Usages

In [7]:
clst = []
for i in unicodes:
    #for each unique code, find its Usage value in master df and add them all together, then append to list
    clst.append(electric.Usage[codes.isin([i])].sum())

### Normalized Usages

In [38]:
nusage = clst/area

4783.1806588384852

### Proportions

In [19]:
tot = sum(clst)
tot

417069167.01770395

In [20]:
tot = sum(clst)
tot

417069167.01770395

In [39]:
atot = 0
for i in nusage:
    if i>0 and i<1000000000000000000:
        atot = atot + i
atot

4783.1806588384852

### Full Building Names

In [8]:
namelst = []
for i in unicodes:
    namelst.append(pc[pc['Property Code'].isin([i])]['Property Name'])
namelst
name = pd.DataFrame(namelst)
name = name.replace(np.nan, '', regex=True)
name = name.reset_index(drop=True)
name =name.sum(axis=1)

### Functions

In [45]:
def normal_usage(meter, date, temperature):
    weekend = pd.to_datetime(date).weekday in [5,6]
    meter_usage = electric_day.ix[meter]
    return meter_usage[(daily_temperature.ix[meter_usage.index]['TEMPERATURE'] > temperature - 4) & (daily_temperature.ix[meter_usage.index] < temperature + 4).squeeze() & (abs(pd.to_datetime(date).dayofyear - pd.to_datetime(meter_usage.index).dayofyear) < 20) & (pd.to_datetime(meter_usage.index).weekday.isin([5,6]) == weekend)]['Usage'].mean()

In [131]:
def percent_difference(meter, date, temperature, usage):
    normal = normal_usage(meter, date, temperature)
    if usage > 100:
        if normal == 0:
            normal = 1
        return 100 * (usage - normal) / normal
    return float('nan')

In [132]:
def percent_diff_lst(meter):
    dates = pd.to_datetime(electric_day.ix[meter].index)
    start = pd.to_datetime('2015-01-01')
    end = pd.to_datetime('2016-04-10')
    electric_day_temp = electric_day.ix[meter][(dates >= start) & (dates <= end)]
    diff_lst = []
    date_lst = []
    for day in electric_day_temp.index:
        temperature = daily_temperature.ix[pd.to_datetime(day).date()]
        usage = electric_day_temp.ix[pd.to_datetime(day).date()]
        diff = percent_difference(meter, day.strftime('%Y-%m-%d'), float(temperature), float(usage))
        diff_lst.append(diff)
        date_lst.append(day)
    return pd.DataFrame(diff_lst, date_lst, ['Percent Difference'])

In [133]:
def get_building_data(name):
    namepc = pc[pc['Property Code'].str.contains(name, na=False)]
    dflst = []
    for i in namepc['Property Code']:
        dflst.append(i)
    df = electric[codes.isin(dflst)]
    df = df.reset_index(drop=True)
    df = df[df != 0]
    return df

In [149]:
def get_anomaly_percent(df, fullname):
    anomdf = percent_diff_lst(fullname)
    anomdf = anomdf.sort_values(by='Percent Difference')
    anomdf['Date'] = anomdf.index
    uselst = []
    templst = []
    predlst = []
    for  i in anomdf.index:
        uselst.append(df[df['Datetime'].isin([i])]['Usage'].sum())
        templst.append(daily_temperature[daily_temperature.index.isin([i])]['TEMPERATURE'].mean())
    anomdf['Temperature'] = templst
    anomdf['Actual Use'] = uselst
    anomdf['Percent Difference'] = abs(anomdf['Percent Difference'])
    return anomdf

###  Cumulative dataframe

In [89]:
data = {'Code': unicodes,'Building': name, 'Usage': clst, 'Area': area, 'Year': age, 'Normalized Usage': nusage}

In [90]:
df = pd.DataFrame(data)

In [91]:
df_cumulative = df.copy()

In [92]:
pcnt_lst = []
for i in df_cumulative['Usage']:
    pcnt_lst.append(i/tot)

In [93]:
df_cumulative['Cumulative prop'] = pcnt_lst

In [94]:
apcnt_lst = []
for i in df_cumulative['Normalized Usage']:
    apcnt_lst.append(i/atot)

In [95]:
df_cumulative['Normalized area prop'] = apcnt_lst

In [100]:
df_cumulative = df_cumulative.sort_values(by='Normalized area prop', ascending=False)
df_cumulative = df_cumulative[df_cumulative['Normalized area prop'] < 1]

In [103]:
df_cumulative = df_cumulative.reset_index(drop=True)
df_cumulative

Area                                        Building     Code  \
0     8134.27                             MRS Building (CLSC)      A50   
1    25810.61                      South Campus Chiller Plant      F15   
2    64725.18              West Campus Combined Utility Plant     B113   
3    58574.06                              Rockefeller Chapel      D29   
4    68706.33                                Jones Laboratory      D19   
5   121692.71   Donnelley Biological Sciences Learning Center      B36   
6    58781.22                            Accelerator Building      B03   
7   471918.02           Gordon Center for Integrative Science      A86   
8   172407.63                   Cummings Life Sciences Center      A62   
9    64814.03               Armour Clinical Research Building      A18   
10  330906.58                        Medical Campus Parking A      A70   
11  111396.48              Franklin McLean Research Institute      A19   
12  131381.17                                Hinds Laboratory      A08   
13   34701.50  Marjorie B. Kovler Viral Oncology Laboratories      A11   
14   90594.75                          University High School      E11   
15   95682.00                   Max Palevsky Commons B/Center      C33   
16   69196.38                         Steam Plant Power Plant      F02   
17   47087.55  Laboratory for Astrophysics and Space Research      B01   
18   47371.45              Biopsychological Research Building      B08   
19    4121.25                            Stagg Field Building      B34   
20   67021.23           A.J. Carlson Animal Research Facility      A22   
21   34539.90                             Smart Museum of Art      C13   
22  145692.97                             Research Institutes      B07   
23   36878.89                                Anatomy Building      D06   
24  215499.23                       Laird Bell Law Quadrangle      H02   
25   20468.55                                  Goodspeed Hall      D42   
26   66725.07                Social Science Research Building      D36   
27  185814.54                           1155 East 60th Street      H01   
28   80983.71                        6054 South Drexel Avenue      I29   
29   61715.43                                    Eckhart Hall      D16   
..        ...                                             ...      ...   
50   66084.15                            950 East 61st Street      I13   
51  188820.81        Reva and David Logan Center for the Arts      I30   
52    4029.10                       University Service Garage      B29   
53   91969.74                                  Ida Noyes Hall      E09   
54  237120.34                             International House      E13   
55   80488.51                    Chicago Theological Seminary      F16   
56   90125.40                             Edward H. Levi Hall      D20   
57   11176.90                    5733 South University Avenue      D14   
58   59286.96                 Kersten Physics Teaching Center      A07   
59  168656.27                             John Crerar Library      A06   
60  140390.61                        Saieh Hall for Economics      D69   
61  146500.59                         Henry Crown Field House      C02   
62    1479.63                     Laboratory Service Building      F08   
63   73836.96                                                  D07/D08   
64  201402.16                            Burton-Judson Courts      H03   
65   46482.51                                 McGiffert House      E52   
66  102865.32                            Abbott Memorial Hall      A15   
67   76246.42                    5608 South Stony Island Blvd      N11   
68   23941.40                               5482-88 Greenwood      L07   
69   45482.94                     6022-24 South Drexel Avenue      I10   
70    7443.43                    5737 South University Avenue      D48   
71   19320.06                                  Midway Studios      I02   
72   57126.74

### MRS Building

In [173]:
mrs = get_building_data('A50')

In [169]:
mrs_anom = get_anomaly_percent(mrs, 'A50 MRS Building (CLSC) (B1)')

/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [170]:
mrs_anom.Date = pd.to_datetime(mrs_anom.Date).dt.month
mrs_anom = mrs_anom.groupby(by='Date').mean()

In [202]:
mrsdata = [Bar(x=mrs_anom.index, y=mrs_anom['Percent Difference'])]
layout = go.Layout(title='MRS Building', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))

In [203]:
fig = go.Figure(data=mrsdata, layout=layout)
iplot(fig)

### South campus chiller plant 

In [204]:
schiller = get_building_data('F15')

In [186]:
schiller_anom1 = get_anomaly_percent(schiller, 'F15 6035 S. Blackstone (B1)')
schiller_anom2 = get_anomaly_percent(schiller, 'F15 6035 S. Blackstone (B2)')

/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [187]:
schiller_anom1.Date = pd.to_datetime(schiller_anom1.Date).dt.month
schiller_anom1 = schiller_anom1.groupby(by='Date').mean()

schiller_anom2.Date = pd.to_datetime(schiller_anom2.Date).dt.month
schiller_anom2 = schiller_anom2.groupby(by='Date').mean()


In [206]:
schillerdata1 = [Bar(x=schiller_anom1.index, y=schiller_anom1['Percent Difference'])]
layout = go.Layout(title='South Campus Chiller Plant (B1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig1 = go.Figure(data=schillerdata1, layout=layout)
iplot(fig1)

In [207]:
schillerdata2 = [Bar(x=schiller_anom2.index, y=schiller_anom2['Percent Difference'])]
layout = go.Layout(title='South Campus Chiller Plant (B2)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig2 = go.Figure(data=schillerdata2, layout=layout)
iplot(fig2)

### West Campus Combined Utility Plant

In [193]:
wcampus = get_building_data('B113')
wcampus

Meter            Type        Date Start Time  \
0       B113 5615 S. Maryland (B1)  Electric Usage  2014-04-11      23:30   
1       B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      00:00   
2       B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      00:30   
3       B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      01:00   
4       B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      01:30   
5       B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      02:00   
6       B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      02:30   
7       B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      03:00   
8       B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      03:30   
9       B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      04:00   
10      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      04:30   
11      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      05:00   
12      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      05:30   
13      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      06:00   
14      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      06:30   
15      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      07:00   
16      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      07:30   
17      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      08:00   
18      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      08:30   
19      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      09:00   
20      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      09:30   
21      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      10:00   
22      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      10:30   
23      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      11:00   
24      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      11:30   
25      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      12:00   
26      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      12:30   
27      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      13:00   
28      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      13:30   
29      B113 5615 S. Maryland (B1)  Electric Usage  2014-04-12      14:00   
...                            ...             ...         ...        ...   
140118  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      09:00   
140119  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      09:30   
140120  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      10:00   
140121  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      10:30   
140122  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      11:00   
140123  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      11:30   
140124  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      12:00   
140125  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      12:30   
140126  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      13:00   
140127  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      13:30   
140128  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      14:00   
140129  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      14:30   
140130  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      15:00   
140131  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      15:30   
140132  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      16:00   
140133  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      16:30   
140134  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      17:00   
140135  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      17:30   
140136  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      18:00   
140137  B113 5615 S. Maryland (B4)  Electric Usage  2016-04-10      18:30   
140138  B113 5615 S. Mary

In [195]:
wcampus_anom1 = get_anomaly_percent(wcampus, 'B113 5615 S. Maryland (B1)')
wcampus_anom2 = get_anomaly_percent(wcampus, 'B113 5615 S. Maryland (B2)')
wcampus_anom3 = get_anomaly_percent(wcampus, 'B113 5615 S. Maryland (B3)')
wcampus_anom4 = get_anomaly_percent(wcampus, 'B113 5615 S. Maryland (B4)')

/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [196]:
wcampus_anom1.Date = pd.to_datetime(wcampus_anom1.Date).dt.month
wcampus_anom1 = wcampus_anom1.groupby(by='Date').mean()

wcampus_anom2.Date = pd.to_datetime(wcampus_anom2.Date).dt.month
wcampus_anom2 = wcampus_anom2.groupby(by='Date').mean()

wcampus_anom3.Date = pd.to_datetime(wcampus_anom3.Date).dt.month
wcampus_anom3 = wcampus_anom3.groupby(by='Date').mean()

wcampus_anom4.Date = pd.to_datetime(wcampus_anom4.Date).dt.month
wcampus_anom4 = wcampus_anom4.groupby(by='Date').mean()

In [285]:
wcampusdata1 = [Bar(x=wcampus_anom1.index, y=wcampus_anom1['Percent Difference'])]
layout1 = go.Layout(title='West Campus Utility Plant (B1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig1 = go.Figure(data=wcampusdata1, layout=layout1)
iplot(fig1)

wcampusdata2 = [Bar(x=wcampus_anom2.index, y=wcampus_anom2['Percent Difference'])]
layout2 = go.Layout(title='West Campus Utility Plant (B2)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig2 = go.Figure(data=wcampusdata2, layout=layout2)
iplot(fig2)

wcampusdata3 = [Bar(x=wcampus_anom3.index, y=wcampus_anom3['Percent Difference'])]
layout3 = go.Layout(title='West Campus Utility Plant (B3)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig3 = go.Figure(data=wcampusdata3, layout=layout3)
iplot(fig3)

wcampusdata4 = [Bar(x=wcampus_anom4.index, y=wcampus_anom4['Percent Difference'])]
layout4 = go.Layout(title='West Campus Utility Plant (B4)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig4 = go.Figure(data=wcampusdata4, layout=layout4)
iplot(fig4)


### Rockefeller Chapel

In [215]:
rock = get_building_data('D29')
rock.Meter.unique()

array(['D29 Rockefeller Chapel (B1)', 'D29 Rockefeller Chapel (U1)'], dtype=object)

In [233]:
rock_anom1 = get_anomaly_percent(rock, 'D29 Rockefeller Chapel (B1)')
rock_anom2 = get_anomaly_percent(rock, 'D29 Rockefeller Chapel (U1)')

/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [234]:
rock_anom1.Date = pd.to_datetime(rock_anom1.Date).dt.month
rock_anom2.Date = pd.to_datetime(rock_anom2.Date).dt.month

rock_anom1 = rock_anom1.groupby(by='Date').mean()
rock_anom2 = rock_anom2.groupby(by='Date').mean()

In [284]:
rockdata1 = [Bar(x=rock_anom1.index, y=rock_anom1['Percent Difference'])]
layout1 = go.Layout(title='Rockefeller Chapel (B1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig1 = go.Figure(data=rockdata1, layout=layout1)
iplot(fig1)

rockdata2 = [Bar(x=rock_anom2.index, y=rock_anom2['Percent Difference'])]
layout2 = go.Layout(title='Rockefeller Chapel (U1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig2 = go.Figure(data=rockdata2, layout=layout2)
iplot(fig2)

### Jones Lab

In [237]:
jones = get_building_data('D19') 
jones.Meter.unique()

array(['D19 Jones Laboratory (B1)', 'D19 Jones S/B Searle (B2)',
       'D19 Jones S/B Searle (B1)', 'D19 Jones Laboratory (B2)',
       'D19 Jones Laboratory (U1)'], dtype=object)

In [251]:
jones_anom1 = get_anomaly_percent(jones, 'D19 Jones Laboratory (B1)')
jones_anom2 = get_anomaly_percent(jones, 'D19 Jones Laboratory (B2)')
jones_anom3 = get_anomaly_percent(jones, 'D19 Jones Laboratory (U1)')


/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [252]:
jones_anom1.Date = pd.to_datetime(jones_anom1.Date).dt.month
jones_anom2.Date = pd.to_datetime(jones_anom2.Date).dt.month
jones_anom3.Date = pd.to_datetime(jones_anom3.Date).dt.month

jones_anom1 = jones_anom1.groupby(by='Date').mean()
jones_anom2 = jones_anom2.groupby(by='Date').mean()
jones_anom3 = jones_anom3.groupby(by='Date').mean()


In [283]:
jonesdata1 = [Bar(x=jones_anom1.index, y=jones_anom1['Percent Difference'])]
layout1 = go.Layout(title='Jones Laboratory (B1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig1 = go.Figure(data=jonesdata1, layout=layout1)
iplot(fig1)

jonesdata2 = [Bar(x=jones_anom2.index, y=jones_anom2['Percent Difference'])]
layout2 = go.Layout(title='Jones Laboratory (B2)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig2 = go.Figure(data=jonesdata2, layout=layout2)
iplot(fig2)

### Donnelley

In [257]:
bslc = get_building_data('B36')
bslc.Meter.unique()

array(['B36 BSLC (B1)', 'B36 BSLC (R3)', 'B36 BSLC (U1)', 'B36 BSLC (B2)',
       'B36 BSLC (U3)', 'B36 BSLC (U4)', 'B36 BSLC (U2)'], dtype=object)

In [276]:
bslc_anom1 = get_anomaly_percent(bslc,'B36 BSLC (B1)')
bslc_anom2 = get_anomaly_percent(bslc,'B36 BSLC (U1)')
bslc_anom3 = get_anomaly_percent(bslc,'B36 BSLC (B2)')
bslc_anom4 = get_anomaly_percent(bslc,'B36 BSLC (U3)')


/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [278]:
bslc_anom1.Date = pd.to_datetime(bslc_anom1.Date).dt.month
bslc_anom2.Date = pd.to_datetime(bslc_anom2.Date).dt.month
bslc_anom3.Date = pd.to_datetime(bslc_anom3.Date).dt.month
bslc_anom4.Date = pd.to_datetime(bslc_anom4.Date).dt.month


bslc_anom1 = bslc_anom1.groupby(by='Date').mean()
bslc_anom2 = bslc_anom2.groupby(by='Date').mean()
bslc_anom3 = bslc_anom3.groupby(by='Date').mean()
bslc_anom4 = bslc_anom4.groupby(by='Date').mean()


In [286]:
bslcdata1 = [Bar(x=bslc_anom1.index, y=bslc_anom1['Percent Difference'])]
layout1 = go.Layout(title='B36 BSLC (B1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig1 = go.Figure(data=bslcdata1, layout=layout1)
iplot(fig1)

bslcdata2 = [Bar(x=bslc_anom2.index, y=bslc_anom2['Percent Difference'])]
layout2 = go.Layout(title='B36 BSLC (U1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig2 = go.Figure(data=bslcdata2, layout=layout2)
iplot(fig2)

bslcdata3 = [Bar(x=bslc_anom3.index, y=bslc_anom3['Percent Difference'])]
layout3 = go.Layout(title='B36 BSLC (B2)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig3 = go.Figure(data=bslcdata3, layout=layout3)
iplot(fig3)

bslcdata4 = [Bar(x=bslc_anom4.index, y=bslc_anom4['Percent Difference'])]
layout4 = go.Layout(title='B36 BSLC (U3)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig4 = go.Figure(data=bslcdata4, layout=layout4)
iplot(fig4)

### Accelerator Building

In [288]:
acc = get_building_data('B03')
acc.Meter.unique()

array(['B03 Accelerator Building (B1)', 'B03 Accelerator Building (B2)'], dtype=object)

In [290]:
acc_anom1 = get_anomaly_percent(acc, 'B03 Accelerator Building (B1)')
acc_anom2 = get_anomaly_percent(acc, 'B03 Accelerator Building (B2)')

/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [291]:
acc_anom1.Date = pd.to_datetime(acc_anom1.Date).dt.month
acc_anom2.Date = pd.to_datetime(acc_anom2.Date).dt.month

acc_anom1 = acc_anom1.groupby(by='Date').mean()
acc_anom2 = acc_anom2.groupby(by='Date').mean()

In [293]:
accdata1 = [Bar(x=acc_anom1.index, y=acc_anom1['Percent Difference'])]
layout1 = go.Layout(title='Accelerator Building (B1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig1 = go.Figure(data=accdata1, layout=layout1)
iplot(fig1)

accdata2 = [Bar(x=acc_anom2.index, y=acc_anom2['Percent Difference'])]
layout2 = go.Layout(title='Accelerator Building (B2)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig2 = go.Figure(data=accdata2, layout=layout2)
iplot(fig2)

### GCIS

In [295]:
gcis = get_building_data('A86')
gcis.Meter.unique()

array(['A86 GCIS (B6)', 'A86 GCIS (B7)', 'A86 GCIS (B8)', 'A86 GCIS (B9)',
       'A86 GCIS (B10)', 'A86 GCIS (B11)', 'A86 GCIS (B12)',
       'A86 GCIS (U1)'], dtype=object)

In [304]:
gcis_anom1 = get_anomaly_percent(gcis, 'A86 GCIS (B6)')
gcis_anom2 = get_anomaly_percent(gcis, 'A86 GCIS (B7)')
gcis_anom3 = get_anomaly_percent(gcis, 'A86 GCIS (B8)')
gcis_anom4 = get_anomaly_percent(gcis, 'A86 GCIS (B9)')
gcis_anom5 = get_anomaly_percent(gcis, 'A86 GCIS (B10)')
gcis_anom6 = get_anomaly_percent(gcis, 'A86 GCIS (B12)')
gcis_anom7 = get_anomaly_percent(gcis, 'A86 GCIS (U1)')

/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [307]:
gcis_anom1.Date = pd.to_datetime(gcis_anom1.Date).dt.month
gcis_anom2.Date = pd.to_datetime(gcis_anom2.Date).dt.month
gcis_anom3.Date = pd.to_datetime(gcis_anom3.Date).dt.month
gcis_anom4.Date = pd.to_datetime(gcis_anom4.Date).dt.month
gcis_anom5.Date = pd.to_datetime(gcis_anom5.Date).dt.month
gcis_anom6.Date = pd.to_datetime(gcis_anom6.Date).dt.month
gcis_anom7.Date = pd.to_datetime(gcis_anom7.Date).dt.month


gcis_anom1 = gcis_anom1.groupby(by='Date').mean()
gcis_anom2 = gcis_anom2.groupby(by='Date').mean()
gcis_anom3 = gcis_anom3.groupby(by='Date').mean()
gcis_anom4 = gcis_anom4.groupby(by='Date').mean()
gcis_anom5 = gcis_anom5.groupby(by='Date').mean()
gcis_anom6 = gcis_anom6.groupby(by='Date').mean()
gcis_anom7 = gcis_anom7.groupby(by='Date').mean()


In [309]:
gcisdata1 = [Bar(x=gcis_anom1.index, y=gcis_anom1['Percent Difference'])]
layout1 = go.Layout(title='A86 GCIS (B6)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig1 = go.Figure(data=gcisdata1, layout=layout1)
iplot(fig1)

gcisdata2 = [Bar(x=gcis_anom2.index, y=gcis_anom2['Percent Difference'])]
layout2 = go.Layout(title='A86 GCIS (B7)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig2 = go.Figure(data=gcisdata2, layout=layout2)
iplot(fig2)

gcisdata3 = [Bar(x=gcis_anom3.index, y=gcis_anom3['Percent Difference'])]
layout3 = go.Layout(title='A86 GCIS (B8)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig3 = go.Figure(data=gcisdata3, layout=layout3)
iplot(fig3)

gcisdata4 = [Bar(x=gcis_anom4.index, y=gcis_anom4['Percent Difference'])]
layout4 = go.Layout(title='A86 GCIS (B9)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig4 = go.Figure(data=gcisdata4, layout=layout4)
iplot(fig4)

gcisdata5 = [Bar(x=gcis_anom5.index, y=gcis_anom5['Percent Difference'])]
layout5 = go.Layout(title='A86 GCIS (B10)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig5 = go.Figure(data=gcisdata5, layout=layout5)
iplot(fig5)

gcisdata6 = [Bar(x=gcis_anom6.index, y=gcis_anom6['Percent Difference'])]
layout6 = go.Layout(title='A86 GCIS (B12)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig6 = go.Figure(data=gcisdata6, layout=layout6)
iplot(fig6)

gcisdata7 = [Bar(x=gcis_anom7.index, y=gcis_anom7['Percent Difference'])]
layout7 = go.Layout(title='A86 GCIS (U1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig7 = go.Figure(data=gcisdata7, layout=layout7)
iplot(fig7)

### Cummings

In [311]:
cummings = get_building_data('A62')
cummings.Meter.unique()

array(['A62 Cummings Life Sciences (B1)',
       'A62 Cummings Life Sciences (B3)',
       'A62 Cummings Life Sciences (B5)',
       'A62 Cummings Life Sciences (B4)', 'A62 Cummings Life Sciences (B6)'], dtype=object)

In [323]:
cummings_anom1 = get_anomaly_percent(cummings, 'A62 Cummings Life Sciences (B1)')
cummings_anom2 = get_anomaly_percent(cummings, 'A62 Cummings Life Sciences (B3)')
cummings_anom3 = get_anomaly_percent(cummings, 'A62 Cummings Life Sciences (B5)')
cummings_anom4 = get_anomaly_percent(cummings, 'A62 Cummings Life Sciences (B6)')


/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [324]:
cummings_anom1.Date = pd.to_datetime(cummings_anom1.Date).dt.month
cummings_anom2.Date = pd.to_datetime(cummings_anom2.Date).dt.month
cummings_anom3.Date = pd.to_datetime(cummings_anom3.Date).dt.month
cummings_anom4.Date = pd.to_datetime(cummings_anom4.Date).dt.month

cummings_anom1 = cummings_anom1.groupby(by='Date').mean()
cummings_anom2 = cummings_anom2.groupby(by='Date').mean()
cummings_anom3 = cummings_anom3.groupby(by='Date').mean()
cummings_anom4 = cummings_anom4.groupby(by='Date').mean()

In [326]:
cummingsdata1 = [Bar(x=cummings_anom1.index, y=cummings_anom1['Percent Difference'])]
layout1 = go.Layout(title='Cummings Life Sciences (B1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig1 = go.Figure(data=cummingsdata1, layout=layout1)
iplot(fig1)

cummingsdata2 = [Bar(x=cummings_anom2.index, y=cummings_anom2['Percent Difference'])]
layout2 = go.Layout(title='Cummings Life Sciences (B3)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig2 = go.Figure(data=cummingsdata2, layout=layout2)
iplot(fig2)

cummingsdata3 = [Bar(x=cummings_anom3.index, y=cummings_anom3['Percent Difference'])]
layout3 = go.Layout(title='Cummings Life Sciences (B5)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig3 = go.Figure(data=cummingsdata3, layout=layout3)
iplot(fig3)

cummingsdata4 = [Bar(x=cummings_anom4.index, y=cummings_anom4['Percent Difference'])]
layout4 = go.Layout(title='Cummings Life Sciences (B6)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig4 = go.Figure(data=cummingsdata4, layout=layout4)
iplot(fig4)

### Armour Clinical Research Institute

In [328]:
acri = get_building_data('A18')
acri.Meter.unique()

array(['A18 Armour Research (B1)'], dtype=object)

In [329]:
acri_anom = get_anomaly_percent(acri, 'A18 Armour Research (B1)')

/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [331]:
acri_anom.Date = pd.to_datetime(acri_anom.Date).dt.month
acri_anom = acri_anom.groupby(by='Date').mean()

In [333]:
acridata= [Bar(x=acri_anom.index, y=acri_anom['Percent Difference'])]
layout1 = go.Layout(title='A18 Armour Research (B1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig1 = go.Figure(data=acridata, layout=layout1)
iplot(fig1)

### Medical Campus Parking A

In [335]:
mcp = get_building_data('A70')
mcp.Meter.unique()


array(['A70 UC Hospital Chiller (B1)', 'A70 UC Hospital Chiller (B2)'], dtype=object)

In [336]:
mcp_anom1 = get_anomaly_percent(mcp, 'A70 UC Hospital Chiller (B1)')
mcp_anom2 = get_anomaly_percent(mcp, 'A70 UC Hospital Chiller (B2)')


/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [339]:
mcp_anom1.Date = pd.to_datetime(mcp_anom1.Date).dt.month
mcp_anom2.Date = pd.to_datetime(mcp_anom2.Date).dt.month

mcp_anom1 = mcp_anom1.groupby(by='Date').mean()
mcp_anom2 = mcp_anom2.groupby(by='Date').mean()

In [340]:
mcpdata1 = [Bar(x=mcp_anom1.index, y=mcp_anom1['Percent Difference'])]
layout1 = go.Layout(title='Medical Campus Parking (B1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig1 = go.Figure(data=mcpdata1, layout=layout1)
iplot(fig1)

mcpdata2 = [Bar(x=mcp_anom2.index, y=mcp_anom2['Percent Difference'])]
layout2 = go.Layout(title='Medical Campus Parking (B2)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig2 = go.Figure(data=mcpdata2, layout=layout2)
iplot(fig2)

### Franklin McLean Research Institute

In [343]:
fmrl = get_building_data('A19')
fmrl.Meter.unique()

array(['A19 Franklin Institute (B2)', 'A19 Franklin Institute (B3)',
       'A19 Franklin Institute (U1)', 'A19 Franklin Institute (U2)',
       'A19 Franklin Institute (U3)'], dtype=object)

In [353]:
fmrl_anom1 = get_anomaly_percent(fmrl, 'A19 Franklin Institute (B3)')
fmrl_anom2 = get_anomaly_percent(fmrl, 'A19 Franklin Institute (U1)')
fmrl_anom3 = get_anomaly_percent(fmrl, 'A19 Franklin Institute (U2)')
fmrl_anom4 = get_anomaly_percent(fmrl, 'A19 Franklin Institute (U3)')


/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [354]:
fmrl_anom1.Date = pd.to_datetime(fmrl_anom1.Date).dt.month
fmrl_anom2.Date = pd.to_datetime(fmrl_anom2.Date).dt.month
fmrl_anom3.Date = pd.to_datetime(fmrl_anom3.Date).dt.month
fmrl_anom4.Date = pd.to_datetime(fmrl_anom4.Date).dt.month

fmrl_anom1 = fmrl_anom1.groupby(by='Date').mean()
fmrl_anom2 = fmrl_anom2.groupby(by='Date').mean()
fmrl_anom3 = fmrl_anom3.groupby(by='Date').mean()
fmrl_anom4 = fmrl_anom4.groupby(by='Date').mean()

In [357]:
fmrldata1 = [Bar(x=fmrl_anom1.index, y=fmrl_anom1['Percent Difference'])]
layout1 = go.Layout(title='A19 Franklin Institute (B3)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig1 = go.Figure(data=fmrldata1, layout=layout1)
iplot(fig1)

fmrldata2 = [Bar(x=fmrl_anom2.index, y=fmrl_anom2['Percent Difference'])]
layout2 = go.Layout(title='A19 Franklin Institute (U1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig2 = go.Figure(data=fmrldata2, layout=layout2)
iplot(fig2)

fmrldata3 = [Bar(x=fmrl_anom3.index, y=fmrl_anom3['Percent Difference'])]
layout3 = go.Layout(title='A19 Franklin Institute (U2)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig3 = go.Figure(data=fmrldata3, layout=layout3)
iplot(fig3)

fmrldata4 = [Bar(x=fmrl_anom4.index, y=fmrl_anom4['Percent Difference'])]
layout4 = go.Layout(title='A19 Franklin Institute (U3)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig4 = go.Figure(data=fmrldata4, layout=layout4)
iplot(fig4)


### Hinds Lab

In [359]:
hinds = get_building_data('A08')
hinds.Meter.unique()

array(['A08 Hinds Laboratories (R1)', 'A08 Hinds Laboratories (B2)',
       'A08 Hinds Laboratories (B1)', 'A08 Hinds Laboratories (U1)',
       'A08 Hinds Laboratories (U2)'], dtype=object)

In [370]:
hinds_anom1 = get_anomaly_percent(hinds, 'A08 Hinds Laboratories (B2)')
hinds_anom2 = get_anomaly_percent(hinds, 'A08 Hinds Laboratories (B1)')
hinds_anom3 = get_anomaly_percent(hinds, 'A08 Hinds Laboratories (U1)')
hinds_anom4 = get_anomaly_percent(hinds, 'A08 Hinds Laboratories (U2)')

/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [371]:
hinds_anom1.Date = pd.to_datetime(hinds_anom1.Date).dt.month
hinds_anom2.Date = pd.to_datetime(hinds_anom2.Date).dt.month
hinds_anom3.Date = pd.to_datetime(hinds_anom3.Date).dt.month
hinds_anom4.Date = pd.to_datetime(hinds_anom4.Date).dt.month

hinds_anom1 = hinds_anom1.groupby(by='Date').mean()
hinds_anom2 = hinds_anom2.groupby(by='Date').mean()
hinds_anom3 = hinds_anom3.groupby(by='Date').mean()
hinds_anom4 = hinds_anom4.groupby(by='Date').mean()

In [373]:
hindsdata1 = [Bar(x=hinds_anom1.index, y=hinds_anom1['Percent Difference'])]
layout1 = go.Layout(title='Hinds Laboratory (B2)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig1 = go.Figure(data=hindsdata1, layout=layout1)
iplot(fig1)

hindsdata2 = [Bar(x=hinds_anom2.index, y=hinds_anom2['Percent Difference'])]
layout2 = go.Layout(title='Hinds Laboratory (B1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig2 = go.Figure(data=hindsdata2, layout=layout2)
iplot(fig2)

hindsdata3 = [Bar(x=hinds_anom3.index, y=hinds_anom3['Percent Difference'])]
layout3 = go.Layout(title='Hinds Laboratory (U1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig3 = go.Figure(data=hindsdata3, layout=layout3)
iplot(fig3)

hindsdata4 = [Bar(x=hinds_anom4.index, y=hinds_anom4['Percent Difference'])]
layout4 = go.Layout(title='Hinds Laboratory (U3)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig4 = go.Figure(data=hindsdata4, layout=layout4)
iplot(fig4)



### Kovler Lab

In [384]:
kovler = get_building_data('A11')
kovler.Meter.unique()

array(['A11 Kovler Laboratories (B1)'], dtype=object)

In [385]:
kovler_anom = get_anomaly_percent(kovler, 'A11 Kovler Laboratories (B1)')

/Users/abisolaolawale/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



In [386]:
kovler_anom.Date = pd.to_datetime(kovler_anom.Date).dt.month
kovler_anom = kovler_anom.groupby(by='Date').mean()

In [387]:
kovlerdata = [Bar(x=kovler_anom.index, y=kovler_anom['Percent Difference'])]
layout1 = go.Layout(title='Kovler Laboratories (B1)', xaxis=dict(title='Month'), yaxis=dict(title='Percent Difference'))
fig1 = go.Figure(data=kovlerdata, layout=layout1)
iplot(fig1)